In [38]:
from mainnet_launch.pages.risk_metrics.exit_liqudity_quotes import (
    _pick_a_quote_batch,
    _load_full_quote_batch_df,
    _augment_swap_quotes_df,
)

import plotly.express as px
import streamlit as st
import plotly.io as pio
import pandas as pd

pio.templates.default = "plotly_white"

from mainnet_launch.constants import *

from mainnet_launch.database.schema.full import Destinations, AutopoolDestinations, Tokens, SwapQuote
from mainnet_launch.database.schema.postgres_operations import (
    get_full_table_as_df,
    get_highest_value_in_field_where,
)

from mainnet_launch.pages.risk_metrics.drop_down import render_pick_chain_and_base_asset_dropdown


chain, base_asset, _ = render_pick_chain_and_base_asset_dropdown()
chain, base_asset = ETH_CHAIN, DOLA
quote_batch_number = 34
full_quote_batch_df = _load_full_quote_batch_df(quote_batch_number)

st.write("Options for this quote batch:")
# st.dataframe(full_quote_batch_df.groupby(["chain_id", "base_asset"]).size().reset_index(name="count"))

swap_quotes_df = full_quote_batch_df[
    (full_quote_batch_df["chain_id"] == chain.chain_id) & (full_quote_batch_df["base_asset"] == base_asset(chain))
].reset_index(drop=True)

_augment_swap_quotes_df(swap_quotes_df)
swap_quotes_df

2025-08-12 11:15:26.387 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 11:15:26.388 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 11:15:26.388 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 11:15:26.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 11:15:26.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 11:15:26.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 11:15:26.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-12 11:15:26.390 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

,chain_id,base_asset,api_name,sell_token_address,buy_token_address,scaled_amount_in,scaled_amount_out,pools_blacklist,percent_exclude_threshold,aggregator_name,datetime_received,quote_batch,size_factor,id,buy_token_symbol,sell_token_symbol,effective_price,reference_price,label,slippage_bps
0,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0xb45ad160634c528Cc3D2926d9807104FA3157305,0x865377367054516e17014CcdED1e7d814EDC9ce4,200000.0,2.298219e+05,['0x6691dbb44154a9f23f8357c56fc9ff5548a8bdc4'],50.0,Odos,2025-08-12 16:40:44.587848+00:00,34,absolute,19fd689a-fd73-4ca4-b2ef-a180ac6ff17e,DOLA,sDOLA,1.149110,1.149110,sDOLA - odos,-0.01
1,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0x57aB1E0003F623289CD798B1824Be09a793e4Bec,0x865377367054516e17014CcdED1e7d814EDC9ce4,400000.0,3.965451e+05,['0x6691dbb44154a9f23f8357c56fc9ff5548a8bdc4'],50.0,Odos,2025-08-12 16:40:52.315309+00:00,34,absolute,37f3738b-7ad0-432c-a636-98631e8e0780,DOLA,reUSD,0.991363,0.992163,reUSD - odos,-8.06
2,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0x57aB1E0003F623289CD798B1824Be09a793e4Bec,0x865377367054516e17014CcdED1e7d814EDC9ce4,100000.0,9.917257e+04,None,50.0,Odos,2025-08-12 16:43:00.425063+00:00,34,absolute,52954a78-6f59-40ba-91d2-4663fef7b248,DOLA,reUSD,0.991726,0.992163,reUSD - tokemak,-4.40
3,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0x15700B564Ca08D9439C58cA5053166E8317aa138,0x865377367054516e17014CcdED1e7d814EDC9ce4,200000.0,2.003129e+05,None,50.0,Odos,2025-08-12 16:43:06.698511+00:00,34,absolute,d8aed3eb-2357-4e5a-82a3-973c26135ea6,DOLA,deUSD,1.001565,1.002169,deUSD - tokemak,-6.04
4,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0x57aB1E0003F623289CD798B1824Be09a793e4Bec,0x865377367054516e17014CcdED1e7d814EDC9ce4,200000.0,1.983213e+05,['0x6691dbb44154a9f23f8357c56fc9ff5548a8bdc4'],50.0,Odos,2025-08-12 16:40:42.754938+00:00,34,absolute,605062e5-c065-41e6-adf6-a0e9f2b31d06,DOLA,reUSD,0.991606,0.992163,reUSD - odos,-5.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0xBC6DA0FE9aD5f3b0d58160288917AA56653660E9,0x865377367054516e17014CcdED1e7d814EDC9ce4,100000.0,9.972303e+04,None,50.0,Odos,2025-08-12 16:43:02.990613+00:00,34,absolute,05ef6f42-c693-43a8-b5db-02046f91d29f,DOLA,alUSD,0.997230,0.997508,alUSD - tokemak,-2.78
128,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,odos,0x9D39A5DE30e57443BfF2A8307A4256c8797A3497,0x865377367054516e17014CcdED1e7d814EDC9ce4,1000000.0,1.189152e+06,['0x6691dbb44154a9f23f8357c56fc9ff5548a8bdc4'],50.0,Odos,2025-08-12 16:41:23.272728+00:00,34,absolute,ecee4953-8587-493a-bd04-47b4f83bcbdb,DOLA,sUSDe,1.189152,1.192679,sUSDe - odos,-29.58
129,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0xb45ad160634c528Cc3D2926d9807104FA3157305,0x865377367054516e17014CcdED1e7d814EDC9ce4,1000000.0,1.149110e+06,None,50.0,4626,2025-08-12 16:44:19.174530+00:00,34,absolute,637e6195-10c7-4568-bcac-d47f0e5d08c8,DOLA,sDOLA,1.149110,1.149110,sDOLA - tokemak,0.00
130,1,0x865377367054516e17014CcdED1e7d814EDC9ce4,tokemak,0x0655977FEb2f289A4aB78af67BAB0d17aAb84367,0x865377367054516e17014CcdED1e7d814EDC9ce4,1000000.0,1.060914e+06,None,50.0,4626,2025-08-12 16:44:11.496365+00:00,34,absolute,4888b01a-34e8-4d78-9ffb-4e6372fcdaa6,DOLA,scrvUSD,1.060914,1.061444,scrvUSD - tokemak,-5.00


In [ ]:
for api_name in ["odos", "tokemak"]:
    local_df = swap_quotes_df[swap_quotes_df["api_name"] == api_name].copy()
    local_df = local_df.sort_values(["label", "scaled_amount_in"])

    fig = px.line(
        local_df,
        x="scaled_amount_in",
        y="slippage_bps",
        color="label",
        markers=True,  # adds markers
    )

    fig.update_traces(
        mode="lines+markers",  # show both
        line=dict(dash="dash"),  # dashed lines
        marker_symbol="x",  # 'x marks the spot'
        marker_size=8,
    )

    fig.update_layout(
        title=f"{api_name}: Slippage vs Amount Sold",
        xaxis_title="scaled_amount_in",
        yaxis_title="slippage_bps",
    )
    fig.show()

In [40]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.templates.default = "plotly_white"

# Sorted symbols
symbol_options = sorted(swap_quotes_df["sell_token_symbol"].dropna().unique().tolist())
factors = ["absolute", "portion"]
factor_symbol = {"absolute": "circle", "portion": "x"}

# Stable colors per label across all charts
all_labels = sorted(swap_quotes_df["label"].dropna().unique().tolist())
palette = px.colors.qualitative.Plotly
label_color = {lbl: palette[i % len(palette)] for i, lbl in enumerate(all_labels)}

for selected_symbol in symbol_options:
    fig = go.Figure()

    for size_factor in factors:
        df_factor = (
            swap_quotes_df[
                (swap_quotes_df["sell_token_symbol"] == selected_symbol)
                & (swap_quotes_df["size_factor"] == size_factor)
            ]
            .groupby(["label", "scaled_amount_in"], as_index=False)["effective_price"]
            .median()
        )
        if df_factor.empty:
            continue

        # One trace per (label, factor) to overlay both factors
        for label, df_lab in df_factor.groupby("label", sort=True):
            fig.add_trace(
                go.Scatter(
                    x=df_lab["scaled_amount_in"],
                    y=df_lab["effective_price"],
                    mode="markers",
                    name=f"{label} ({size_factor})",
                    legendgroup=label,  # group legend entries by label
                    marker=dict(
                        symbol=factor_symbol.get(size_factor, "circle"),
                        size=7,
                        color=label_color.get(label),
                        line=dict(width=0.5),
                    ),
                    hovertemplate=(
                        "label=%{text}<br>"
                        "factor=" + size_factor + "<br>"
                        "scaled_amount_in=%{x}<br>"
                        "effective_price=%{y}<extra></extra>"
                    ),
                    text=[label] * len(df_lab),
                )
            )

    fig.update_xaxes(title_text="scaled_amount_in")
    fig.update_yaxes(title_text="effective_price")
    fig.update_layout(
        title_text=f"Slippage vs Amount — {selected_symbol} (absolute & portion overlaid)",
        margin=dict(t=60, r=20, b=20, l=50),
        legend_title_text="label (factor)",
    )

    # st.plotly_chart(fig, use_container_width=True)
    fig.show()